In [3]:
import pandas as pd
import random

I selected this dataset from Arlington county. It's every transaction at a parking meter in Arlington from the last 10 years.  It's way too big for my computer (8GB, 36MM records), so I'm taking a random 10% sample

I'm not including the raw data in my repo, it can be downloaded here: https://data.arlingtonva.us/dataset/148?tab=2 

In [21]:

park = pd.read_csv(
    'ParkingTransactions.txt'
    ,delimiter='|'
    ,skiprows=lambda i: i>0 and random.random() > 0.1
    )

/var/folders/tn/1tqsy6b16xl_cxd8g27h18y00000gp/T/ipykernel_52151/762901319.py:2: DtypeWarning: Columns (2,17) have mixed types. Specify dtype option on import or set low_memory=False.
  park = pd.read_csv(


,ParkingTransactionKey,TransactionSourceCode,MeterId,ZoneNbr,StartDtm,EndDtm,TransactionAmt,PaymentTypeName,TransactionStatusCode,MeterManufacturerName,...,StatePlaneXCrd,StatePlaneYCrd,HandicapInd,TimeRestrictionDsc,MaxHoursCnt,MeterTypeDsc,DollarPerHourRate,ActiveStatusInd,ZoneSpaceCnt,MetroAreaName
0,12,IPS,QUH2717S,54016,2015-12-31 17:03:07,2015-12-31 17:47:50,0,Coins,NaN,IPS,...,11885481,6992462,True,MON - SAT 8AM - 8PM,4,Single,2,True,1,Shirlington
1,17,IPS,316N,54469,2015-12-31 16:46:52,2015-12-31 17:46:52,2,Coins,NaN,IPS,...,11885759,7010487,False,"MON - FRI 8AM - 8PM, SAT 6AM - 12PM",1,Single,2,False,2,Courthouse
2,35,IPS,CHH1405N,54413,2015-12-31 15:26:26,2015-12-31 16:12:02,0,Coins,NaN,IPS,...,11886040,7010256,True,MON - FRI 8AM - 8PM,12,Single,2,False,6,Courthouse
3,38,IPS,CR2108S,54066,2015-12-31 15:12:49,2015-12-31 18:00:00,4,Credit Card,Approved,IPS,...,11895967,6997810,False,MON - SAT 8AM - 8PM,2,Double,2,False,14,Crystal City
4,67,IPS,316N,54469,2015-12-31 13:29:43,2015-12-31 13:57:33,0,Coins,NaN,IPS,...,11885759,7010487,False,"MON - FRI 8AM - 8PM, SAT 6AM - 12PM",1,Single,2,False,2,Courthouse


When I tried to set the datatypes, I got a bunch of errors parsing the date column. 
Here was my first attempt to filter out records that had malformed dates.

In [27]:
def is_valid_date(date_str):
    try:
        pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S')
        return True
    except ValueError:
        return False

# Filter out invalid dates
park_selected_cols = park[park['StartDtm'].apply(is_valid_date)]
invalid_records = park[~park['StartDtm'].apply(is_valid_date)]

print(f"Dropped {len(invalid_records)} rows due to invalid dates.")

Dropped 121067 rows due to invalid dates.


A good practice - I didn't throwaway the bad records, I copied them into a new dataframe so that I could inspect them

and look what I found!

In [28]:
invalid_records

,ParkingTransactionKey,TransactionSourceCode,MeterId,ZoneNbr,StartDtm,EndDtm,TransactionAmt,PaymentTypeName,TransactionStatusCode,MeterManufacturerName,...,StatePlaneXCrd,StatePlaneYCrd,HandicapInd,TimeRestrictionDsc,MaxHoursCnt,MeterTypeDsc,DollarPerHourRate,ActiveStatusInd,ZoneSpaceCnt,MetroAreaName
1766364,24498377,ParkMobile,NaN,54294,2021-03-06 19:09:38.180000000,2021-03-06 20:00:38.180000000,2,VISA,Approved,CALE,...,11883297,7008901,False,MON - SAT 8AM - 8PM,4,Multi,2,False,6,Clarendon
1766365,24498493,ParkMobile,NaN,54266,2021-03-06 18:51:02.953000000,2021-03-06 20:01:02.953000000,2,VISA,Approved,CALE,...,11883361,7009268,False,MON - SAT 8AM - 8PM,2,Multi,2,False,10,Clarendon
1766369,24498530,ParkMobile,NaN,54170,2021-03-06 18:46:32.783000000,2021-03-06 19:42:32.783000000,2,VISA,Approved,CALE,...,11878661,7007343,False,MON - SAT 8AM - 8PM,2,Multi,2,False,5,Ballston
1766370,24498561,ParkMobile,NaN,54159,2021-03-06 18:42:22.040000000,2021-03-06 20:00:22.040000000,2,PayPal,Approved,CALE,...,11879165,7006993,False,MON - SAT 8AM - 8PM,2,Multi,2,False,10,Ballston
1766376,24498595,ParkMobile,NaN,54429,2021-03-06 18:36:58.543000000,2021-03-06 20:00:58.543000000,2,VISA,Approved,CALE,...,11884809,7009964,False,MON - SAT 8AM - 8PM,2,Multi,2,False,4,Courthouse
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3362545,26593572,ParkMobile,NaN,54538,2021-09-20 07:58:33.373000000,2021-09-20 08:58:33.373000000,2,VISA,Approved,CALE,...,11888766,7012214,False,MON - SAT 8AM - 8PM,2,Multi,2,False,4,Rosslyn
3362546,26593637,ParkMobile,NaN,54772,2021-09-20 07:50:31.890000000,2021-09-20 09:20:31.890000000,2,AMEX,Approved,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,False,6,Rosslyn
3362547,26593741,ParkMobile,NaN,54365,2021-09-20 07:18:51.370000000,2021-09-20 09:38:51.370000000,4,VISA,Approved,CALE,...,11893335,7001136,False,MON - FRI 8AM - 8PM,12,Multi,2,False,18,Pentagon City
3362548,26593748,ParkMobile,NaN,54513,2021-09-20 07:15:29.810000000,2021-09-20 11:15:29.810000000,7,VISA,Approved,CALE,...,11888444,7012311,False,MON - SAT 8AM - 8PM,4,Multi,2,False,18,Rosslyn


I definitely don't want to lose all the ParkMobile data, so I just need to filter these records differently

In [ ]:
park_named_cols = park_selected_cols.rename(columns={"ParkingTransactionKey":"id","StartDtm":"start","TransactionAmt":"amt","PaymentTypeName":"type"})

park_named_cols['id'] = park_selected_cols['id'].astype(int)
park_named_cols['month'] = pd.to_datetime(park_selected_cols['start']).dt.month
park_named_cols['day'] = pd.to_datetime(park_selected_cols['start']).dt.day
park_named_cols['year'] = pd.to_datetime(park_selected_cols['start']).dt.year
park_named_cols['amt'] = (park_selected_cols['amt'] * 100).astype(int)
park_named_cols = park_selected_cols.drop(columns=['start'])

park_clean = park_named_cols

In [17]:
park_clean.head

<bound method NDFrame.head of                id  amt         type         day  year
0               1   25        Coins  2015-12-31  2015
1              16   25        Coins  2015-12-31  2015
2              37  175        Coins  2015-12-31  2015
3              79  150  Credit Card  2015-12-31  2015
4              92   25        Coins  2015-12-31  2015
...           ...  ...          ...         ...   ...
3612900  36755071  125  Credit Card  2024-06-24  2024
3612901  36755087  100        Coins  2024-06-24  2024
3612902  36755090   25        Coins  2024-06-24  2024
3612903  36755103  150  Credit Card  2024-06-24  2024
3612904  36755130   25        Coins  2024-06-24  2024

[3491348 rows x 5 columns]>

Let's take a look at the summary stats for the numeric columns.

Pandas was being annoying about displaying the numbers in scientific notation. This lambda function uses an fstring to convert them to whole numbers.

In [29]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)
park_clean.describe()

,id,amt,year
count,3491348,3491348,3491348
mean,18048424,224,2019
std,10678372,190,3
min,1,1,2014
25%,8848786,100,2017
50%,17618474,175,2019
75%,27759821,300,2022
max,36755163,15194,2024
